In [88]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

In [133]:
def makeUrl(search, start_date, end_date, start_pg=1, end_pg=381):
    date_part = "&pd=3&ds=" + start_date + "&de=" + end_date
    return ["https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + date_part + "&sort=2&start=" + str((i - 1) * 10 + 1) for i in range(start_pg, end_pg + 1)]

In [134]:
def get_naver_articles(urls):
    articles, titles, dates, press = [], [], [], []
    
    for url in urls:
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.text, "html.parser")

        article_links = [link.attrs['href'] for link in soup.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")]
        article_titles = [title.text for title in soup.select(".news_tit")]
        article_infos = [info.text for info in soup.select(".info")]

        new_urls = []
        idx = 0

        while idx < len(article_links) - 1:
            if 'news.naver.com' in article_links[idx]:
                # Check if the next link also contains 'news.naver.com'
                if idx + 1 < len(article_links) and 'news.naver.com' in article_links[idx + 1]:
                    new_urls.append(article_links[idx + 1])
                    idx += 2
                else:
                    new_urls.append(article_links[idx])
                    idx += 1
            elif 'news.naver.com' in article_links[idx + 1]:
                new_urls.append(article_links[idx + 1])
                idx += 2
            else:
                new_urls.append(article_links[idx])
                idx += 1

        # 마지막 요소 처리
        if idx == len(article_links) - 1:
            new_urls.append(article_links[idx])

        date_pattern = re.compile(r"\d{4}\.\d{2}\.\d{2}")

        current_dates = []
        current_press = []

        for idx, info in enumerate(article_infos):
            match = date_pattern.search(info)
            if match:
                current_dates.append(match.group(0))
                
                if idx > 0:  # 인덱스 0 앞에는 데이터가 없으므로 idx > 0 조건 추가
                    current_press.append(article_infos[idx - 1])

        articles.extend(new_urls)
        titles.extend(article_titles)
        dates.extend(current_dates)
        press.extend(current_press)

        time.sleep(1)
        
    return articles, titles, dates, press

In [135]:
def get_article_contents(article_urls):
    contents = []
    article_time = []  # 이름 변경
    
    for url in tqdm(article_urls):
        if 'news.naver.com' in url:
            news = requests.get(url, headers=headers)
            soup = BeautifulSoup(news.text, "html.parser")

            content = soup.select("#dic_area") or soup.select("#articeBody")
            cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content)).replace("flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", '')
            contents.append(cleaned_content)

            try:
                date = soup.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span").attrs['data-date-time']
            except AttributeError:
                date = re.sub('<[^>]*>', '', str(soup.select_one("#content > div.end_ct > div > div.article_info > span > em")))
            article_time.append(date)  # 이름 변경
        else:
            contents.append('')
            article_time.append('')
    
    return contents, article_time


In [136]:
# def main():
#     search = input("검색할 키워드를 입력해주세요:")
#     start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
#     end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")

#     urls = makeUrl(search, start_date, end_date)
#     article_urls, article_titles, article_dates, article_press = get_naver_articles(urls)
    
#     contents, article_time = get_article_contents(article_urls)  # 이름 변경
    
#     df = pd.DataFrame({'date': article_dates, 'time': article_time, 'title': article_titles, 'content': contents,'press': article_press, 'link': article_urls })  # 이름 변경
#     df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
#     return df

# if __name__ == "__main__":
#     result_df = main()
#     print(result_df)

In [137]:
search = input("검색할 키워드를 입력해주세요:")
start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")

urls = makeUrl(search, start_date, end_date)
article_urls, article_titles, article_dates, article_press = get_naver_articles(urls)

contents, article_time = get_article_contents(article_urls)  # 이름 변경

# 여기서 각 리스트의 길이를 확인합니다.
print("Length of article_urls:", len(article_urls))
print("Length of article_titles:", len(article_titles))
print("Length of article_dates:", len(article_dates))
print("Length of article_press:", len(article_press))
print("Length of contents:", len(contents))
print("Length of article_time:", len(article_time))

# 길이가 동일한지 확인
if len(article_titles) == len(article_dates)== len(article_press) == len(article_urls) == len(contents):
    df = pd.DataFrame({'date': article_dates, 'time': article_time, 'title': article_titles, 'content': contents,'press': article_press, 'link': article_urls }) 
    df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    print(df)
else:
    print("The lengths of the lists are not equal. Dataframe cannot be created.")



100%|██████████| 3809/3809 [19:30<00:00,  3.25it/s]

Length of article_urls: 3809
Length of article_titles: 3809
Length of article_dates: 3809
Length of article_press: 3809
Length of contents: 3809
Length of article_time: 3809
            date                 time  \
0     2018.01.19                        
1     2018.01.19                        
2     2018.01.19                        
3     2018.01.19                        
4     2018.01.19                        
...          ...                  ...   
3801  2018.01.29                        
3802  2018.01.29                        
3803  2018.01.29                        
3804  2018.01.29                        
3805  2018.01.29  2018-01-29 23:59:53   

                                                  title  \
0       '뉴스룸' 유시민·정재승, 가상화폐 관련 논쟁…"암호화폐는 화폐 아냐" VS "...   
1              [가상화폐 현황] 리플 시세 60% 급등…이오스·퀀텀·이더리움클래식 상승   
2         [암호화폐 시황] 비트코인 2.18%↑·이더리움 3.36%↑...0시 00분 현재   
3                       금감원 직원, 가상화폐 대책 발표 직전에 팔아 수익 올려   
4         가상화폐 선물거래 기대감?빗썸,비트코인15.87%리

In [139]:
df

,date,time,title,content,press,link
0,2018.01.19,,"'뉴스룸' 유시민·정재승, 가상화폐 관련 논쟁…""암호화폐는 화폐 아냐"" VS ""...",,아시아투데이,http://www.asiatoday.co.kr
1,2018.01.19,,[가상화폐 현황] 리플 시세 60% 급등…이오스·퀀텀·이더리움클래식 상승,,한스경제,http://www.hansbiz.co.kr/
2,2018.01.19,,[암호화폐 시황] 비트코인 2.18%↑·이더리움 3.36%↑...0시 00분 현재,,글로벌경제,http://www.getnews.co.kr
3,2018.01.19,,"금감원 직원, 가상화폐 대책 발표 직전에 팔아 수익 올려",,한국정경신문,http://www.kpenews.com
4,2018.01.19,,"가상화폐 선물거래 기대감?빗썸,비트코인15.87%리플47.95%이오스30.7%폭등",,아주경제,http://www.ajunews.com
...,...,...,...,...,...,...
3801,2018.01.29,,[암호화폐 시황] 비트코인 0.58%↑·이더리움 0.22%↑...23시 00분 현재,,글로벌경제,http://www.getnews.co.kr
3802,2018.01.29,,"[가상화폐] '무기명' 마지막 밤... 빗썸, 비트코인 1300만원대 붕괴-리플ㆍ퀀...",,한스경제,http://www.hansbiz.co.kr/
3803,2018.01.29,,"위메프, ""가상화폐로 결제하세요""…'초 단위 가갼 변동 반영이 문제'",,이코노뉴스,http://www.econonews.co.kr/
3804,2018.01.29,,[가상화폐 시세] 리플·퀀텀 제외하고 모두 하락세…비트코인·모네로·이오스...,,이코노뉴스,http://www.econonews.co.kr/


In [79]:
# end_date = '2017.10.09'

In [102]:
#데이터 프레임 저장
filename = '{}_{}_{}.csv'.format(search, start_date.replace(".", ""), end_date.replace(".", ""))
df.to_csv(filename, encoding='utf-8-sig', index=False)

In [55]:
article_urls[70:80]

['http://sports.news.naver.com/column/columnList.nhn?expertId=437',
 'https://sports.news.naver.com/news.nhn?oid=260&aid=0000001176',
 'https://n.news.naver.com/mnews/article/008/0003920735?sid=101',
 'https://n.news.naver.com/mnews/article/374/0000137905?sid=101',
 'http://www.digitaltoday.co.kr/',
 'http://www.etoday.co.kr',
 'http://www.etoday.co.kr',
 'http://www.getnews.co.kr',
 'http://www.koreatimes.com',
 'http://www.getnews.co.kr']

In [27]:
article_titles[3990:4000]

['[가상화폐 시황] 비트코인 -0.29%↓·이더리움 0.67%↑...17시 00분 현재',
 '멕시코 중앙은행 총재 “보증없는 비트코인은 화폐 아니다”',
 '비트코인 억만장자 등극한 스웨덴 개발자…비결은?',
 '[가상화폐 시황] 비트코인 0.14%·이더리움 1.06%↑...18시 00분 현재',
 '[클릭 신간] 힐빌리의 노래 외 2권',
 '"자리가 많지 않습니다"…31일 세종대서 은행 공동 취업설명회',
 '에임하이, 한류스타 이름 딴 가상화폐 내놓는다',
 '늘어나는 가상화폐 범죄',
 '[가상화폐 시황] 비트코인 -0.08%·이더리움 1.76%↑...19시 00분 현재',
 '[가상화폐 시황] 비트코인 0.00%·이더리움 -0.39%↓...20시 00분 현재']

In [141]:
df3 = df

In [145]:
df3

,date,time,title,content,press,link
0,2018.01.19,,"'뉴스룸' 유시민·정재승, 가상화폐 관련 논쟁…""암호화폐는 화폐 아냐"" VS ""...",,아시아투데이,http://www.asiatoday.co.kr
1,2018.01.19,,[가상화폐 현황] 리플 시세 60% 급등…이오스·퀀텀·이더리움클래식 상승,,한스경제,http://www.hansbiz.co.kr/
2,2018.01.19,,[암호화폐 시황] 비트코인 2.18%↑·이더리움 3.36%↑...0시 00분 현재,,글로벌경제,http://www.getnews.co.kr
3,2018.01.19,,"금감원 직원, 가상화폐 대책 발표 직전에 팔아 수익 올려",,한국정경신문,http://www.kpenews.com
4,2018.01.19,,"가상화폐 선물거래 기대감?빗썸,비트코인15.87%리플47.95%이오스30.7%폭등",,아주경제,http://www.ajunews.com
...,...,...,...,...,...,...
3801,2018.01.29,,[암호화폐 시황] 비트코인 0.58%↑·이더리움 0.22%↑...23시 00분 현재,,글로벌경제,http://www.getnews.co.kr
3802,2018.01.29,,"[가상화폐] '무기명' 마지막 밤... 빗썸, 비트코인 1300만원대 붕괴-리플ㆍ퀀...",,한스경제,http://www.hansbiz.co.kr/
3803,2018.01.29,,"위메프, ""가상화폐로 결제하세요""…'초 단위 가갼 변동 반영이 문제'",,이코노뉴스,http://www.econonews.co.kr/
3804,2018.01.29,,[가상화폐 시세] 리플·퀀텀 제외하고 모두 하락세…비트코인·모네로·이오스...,,이코노뉴스,http://www.econonews.co.kr/


In [69]:
len(df1)
len(df2)
len(df3)
print(len(df1)+len(df2)+len(df3))

1054


In [146]:
result = pd.concat([df1, df2, df3], ignore_index=True)
result

,date,time,title,content,press,link
0,2018.01.01,,"[가상화폐] 빗썸거래소, 2018년 비트코인 등 대부분 시세 상승..리플은 하락",,한스경제,http://www.hansbiz.co.kr/
1,2018.01.01,,[암호화폐 시황] 비트코인 1.04%↑·이더리움 0.20%...0시 00분 현재,,글로벌경제,http://www.getnews.co.kr
2,2018.01.01,2018-01-01 00:03:03,중소형펀드·ELS는 투자 포트폴리오에 꼭 쥐어라,\n4대 시중은행 대표 PB들의 투자 조언中·印尼·베트남·러시아 등 신흥국 펀드 가...,A12면 TOP,https://n.news.naver.com/mnews/article/009/000...
3,2018.01.01,,"‘가상화폐’ 빗썸거래소, 2018년 1월 1일 새벽 비트코인-리플코인-비트코인캐...",,톱스타뉴스,http://www.TopStarNews.Net
4,2018.01.01,,2018년 비트코인·비트코인 캐시·대시 등 가상화폐 대부분 상승… 비트코인 ...,,글로벌이코노믹,http://www.g-enews.com/
...,...,...,...,...,...,...
11116,2018.01.29,,[암호화폐 시황] 비트코인 0.58%↑·이더리움 0.22%↑...23시 00분 현재,,글로벌경제,http://www.getnews.co.kr
11117,2018.01.29,,"[가상화폐] '무기명' 마지막 밤... 빗썸, 비트코인 1300만원대 붕괴-리플ㆍ퀀...",,한스경제,http://www.hansbiz.co.kr/
11118,2018.01.29,,"위메프, ""가상화폐로 결제하세요""…'초 단위 가갼 변동 반영이 문제'",,이코노뉴스,http://www.econonews.co.kr/
11119,2018.01.29,,[가상화폐 시세] 리플·퀀텀 제외하고 모두 하락세…비트코인·모네로·이오스...,,이코노뉴스,http://www.econonews.co.kr/


In [147]:
result.to_csv('비트코인_20180101_20180129.csv', encoding='utf-8-sig', index=False)